In [1]:
import pandas as pd
import os
#from Preprocessing.params import *


def get_data(LOCAL_FILEPATH):
    data = pd.read_csv(LOCAL_FILEPATH)
    print(f"data acquired with shape {data.shape}")
    return data



#This is the first 2 days worth of data where general_runs stats are all skewed
def remove_221_rows(data):
    data = data[221:]
    print(f"first 221 rows removed. New shape = {data.shape}")
    return data

## Dropping rows if we have no betting data, and imputing if we have some betting data
def dropping_no_betting_data(data):
    data = data.dropna(subset=['f_bsp', 'f_pm_15m', 'f_pm_10m', 'f_pm_05m' , 'f_pm_03m', 'f_pm_02m', 'f_pm_01m'], how='all')
    def impute_row(row):
        row = row.fillna(method='ffill').fillna(method='bfill')
        return row
    columns_to_impute = ['f_bsp', 'f_pm_15m', 'f_pm_10m', 'f_pm_05m', 'f_pm_03m', 'f_pm_02m', 'f_pm_01m']
    data[columns_to_impute] = data[columns_to_impute].apply(impute_row, axis=1)

    #deleting row that has 0s for all odds for some reason


    print(f"Cleaned up missing odds. New shape = {data.shape}")
    return data

def josh_features(data):

    #Creating country feature
    irish_tracks = [
    "SLIGO", "LIMERICK", "NAVAN", "WEXFORD", "CURRAGH",
    "GALWAY", "KILBEGGAN", "GOWRAN PARK", "BELLEWSTOWN",
    "LISTOWEL", "THURLES", "BALLINROBE", "TRAMORE",
    "LEOPARDSTOWN", "DOWN ROYAL", "ROSCOMMON", "CORK",
    "DUNDALK", "KILLARNEY", "LAYTOWN", "TIPPERARY",
    "FAIRYHOUSE", "NAAS", "DOWNPATRICK", "CLONMEL",
    "PUNCHESTOWN"
]

    data['country'] = data['f_track'].apply(lambda x: 'IRE' if x in irish_tracks else 'GB')

    #Completing f_class for Irish races

    # Calculate mean ratings for each 'f_id' group
    mean_ratings_by_id = data.groupby('f_id')['f_rating_or'].mean()

    # Define the mapping of mean ratings to f_class values
    rating_to_f_class_mapping = {
        (96, float('inf')): 1,
        (86, 96): 2,
        (76, 86): 3,
        (66, 76): 4,
        (56, 66): 5,
        (46, 56): 6,
        (-float('inf'), 46): 7
    }

    # Function to map mean ratings to f_class values
    def map_rating_to_f_class(mean_rating):
        for rating_range, f_class_value in rating_to_f_class_mapping.items():
            if rating_range[0] <= mean_rating <= rating_range[1]:
                return f_class_value

    # Apply the mapping to fill NULL values in 'f_class' column based on mean ratings
    data['f_class'] = data.apply(lambda row: map_rating_to_f_class(mean_ratings_by_id.get(row['f_id'])), axis=1)

    # Now the 'f_class' column should be filled based on the specified mapping using mean ratings

    # Merge the mean ratings back into the original DataFrame based on 'f_id'
    data = data.merge(mean_ratings_by_id, how='left', left_on='f_id', right_index=True)

    # Rename the merged mean rating column for clarity
    data.rename(columns={'f_rating_or_y': 'mean_f_rating_or_race', 'f_rating_or_x' : 'f_rating_or' }, inplace=True)

    # Create official rating vs average rating in the race feature
    data['or_rating_vs_avg_race'] = data['f_rating_or'] - data['mean_f_rating_or_race']

    # Create odds percentage and movement features
    data['15m_odds_prob'] = 1 / data['f_pm_15m']
    data['5m_odds_prob'] = 1 / data['f_pm_05m']
    data['15to5m_odds_move_perc'] = (data['5m_odds_prob'] / data['15m_odds_prob'] - 1)
    data['15to5m_odds_move_raw'] = (data['5m_odds_prob'] - data['15m_odds_prob'])
    print(f"Added Josh features. New shape = {data.shape}")
    return data

def class_or_rating_average(data):
    average_or_rating_class = data.groupby("f_class")['f_rating_or'].mean().reset_index()
    average_or_rating_class.rename(columns={'f_rating_or': 'average_or_rating_class'}, inplace=True)
    data = data.merge(average_or_rating_class, on='f_class')
    data['above_below_official_rating_class'] = data['f_rating_or']- data['average_or_rating_class']
    print(f"Added Oli features 2/4. New shape = {data.shape}")
    return data


def oli_features(data):
    data = data.sort_values(by='f_ko')
    data['PreviousPosition'] = data.groupby('f_horse')['f_place'].shift(fill_value=0)

    data = data.sort_values(by=['f_id', 'pred_isp'])
    data['PredictedRank'] = data.groupby('f_id').cumcount() + 1
    print(f"Added Oli features 4/4. New shape = {data.shape}")
    return data

In [2]:
data = get_data("../raw_data/raw_data_v2.2.csv")
data = remove_221_rows(data)
data = dropping_no_betting_data(data)
    #Dropping line that has 0 for all odds
data = data[data.id != 16157910000382]
data = josh_features(data)
data = class_or_rating_average(data)
data = oli_features(data)

/var/folders/w6/wdkqbmf17b90n_r35p0cb7x00000gn/T/ipykernel_13998/1371766039.py:7: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(LOCAL_FILEPATH)


data acquired with shape (118575, 116)
first 221 rows removed. New shape = (118354, 116)


/var/folders/w6/wdkqbmf17b90n_r35p0cb7x00000gn/T/ipykernel_13998/1371766039.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[columns_to_impute] = data[columns_to_impute].apply(impute_row, axis=1)


Cleaned up missing odds. New shape = (118093, 116)
Added Josh features. New shape = (118092, 123)
Added Oli features 2/4. New shape = (118092, 125)
Added Oli features 4/4. New shape = (118092, 127)


In [3]:
import numpy as np
import pandas as pd

from sklearn import set_config
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

def preprocess_features_v2(data: pd.DataFrame) -> np.ndarray:
    #--------------------------------------------------------------------------#
    print('number of columns: ', len(data.columns))
    ### DROP IRRELEVANT COLUMNS ###
    redundant_columns = [#"id",
                         #"f_id",
                         "f_racetype",
                         #"f_horse",
                         "f_jockey",
                         "f_trainer",
                         "f_rating_hc",
                         "f_lto_pos",
                         "f_bsp",
                         "f_pm_15m",
                         "f_pm_10m",
                         "f_pm_05m",
                         "f_pm_03m",
                         "f_pm_02m",
                         #"f_pm_01m",
                         "f_bsp_p_back",
                         "f_bsp_p_lay",
                         #"f_pm_01m_p_back",
                         "f_pm_01m_p_lay",
                         "f_pm_15m_p_back",
                         "f_pm_15m_p_lay",
                         "general_runs_win_at",
                         "general_runs_win_l200r",
                         "general_runs_win_l50r",
                         "general_runs_win_l16r",
                         "general_runs_at",
                         "general_runs_l200r",
                         "general_runs_l50r",
                         "general_runs_l16r",
                         "sum_bsp_trainer_at",
                         "sum_bsp_jockey_at",
                         "sum_bsp_horse_at",
                         "sum_bsp_trainer_l16r",
                         "sum_bsp_jockey_l16r",
                         "sum_bsp_trainer_l50r",
                         "sum_bsp_jockey_l50r",
                         "sum_bsp_trainer_l200r",
                         "sum_bsp_jockey_l200r",
                         "sum_bsp_horse_l10r",
                         "sum_bsp_horse_l5r",
                         "sum_bsp_horse_l2r",
                         "15to5m_odds_move_perc",
                         "15to5m_odds_move_raw",
                         "15m_odds_prob",
                         "5m_odds_prob"]
    data.drop(columns = redundant_columns, inplace = True)
    print("✅ DROPPED IRRELEVANT COLUMNS")

    #--------------------------------------------------------------------------#

    ### DROP ROWS WITH NULL VALUES IN THESE COLUMNS ###
    data.dropna(axis = 0, inplace = True,
                subset = ["f_pace",
                          "f_stall",
                          "stall_position",
                          "f_going",
                          "f_rating_or",
                          "or_rating_vs_avg_race",
                          "country",
                          "mean_f_rating_or_race",
                          "f_class",
                          "average_or_rating_class",
                          "above_below_official_rating_class",
                          "PreviousPosition",
                          "PredictedRank",
                          ])
    print("✅ DROPPED ROWS WITH NULL VALUES")

    #--------------------------------------------------------------------------#

    ### STRIP SURROUNDING WHITESPACE ###
    data['f_track'] = data['f_track'].str.strip()
    print("✅ WHITESPACE STRIPPED FROM 'f_track'")

    #--------------------------------------------------------------------------#

    ### CONVERT ODDS TO PROBABILITY ###
    def odds_to_prob(x):
        return 1/x
    data['pred_isp'] = data['pred_isp'].apply(odds_to_prob)
    print("✅ ODDS CONVERTED TO PROBABILITY (1/ODDS)")

    #--------------------------------------------------------------------------#

    ### CODE WINNERS AS '1', REST AS '0' ###
    def winner(x):
        if x == 1:
            return 1
        else:
            return 0
    data['f_place'] = data['f_place'].apply(winner)
    print("✅ WINNERS CODED AS '1', REST '0'")

    #--------------------------------------------------------------------------#

    ### 'f_ko' CONVERTED TO DATETIME ###
    data['f_ko'] = data['f_ko'].astype('datetime64[ns]')
    print("✅ 'f_ko' CONVERTED TO DATETIME")

    #--------------------------------------------------------------------------#

    ### ENCODE THE GOING (GROUND CONDITION) ###
    def f_going_coder(x):
        if x == 'FRM':
            return 1
        elif x == 'GTF':
            return 2
        elif x == 'GD' or x == 'GTY' or x == 'STD':
            return 3
        elif x == 'YLD' or x == 'YTS' or x == 'STSL' or x == 'GTS':
            return 4
        elif x == 'SFT':
            return 5
        elif x == 'HVY' or x == 'HTS':
            return 6
    data['f_going'] = data['f_going'].apply(f_going_coder)
    print("✅ TRACK CONDITIONS ORDINALLY ENCODED")

    #--------------------------------------------------------------------------#

    ### MINMAX SCALE NUMERIC FEATURES ###
    set_config(transform_output = "pandas")
    numeric_features = ["f_going",
                        "average_or_rating_class",
                        "above_below_official_rating_class",
                        "PreviousPosition",
                        "PredictedRank",
                        "f_distance",
                        "f_class",
                        "f_age",
                        "f_pace",
                        "f_weight",
                        "f_runners",
                        "f_rating_or",
                        "mean_f_rating_or_race",
                        "or_rating_vs_avg_race",
                        "f_rating_rbd",
                        "f_stall",
                        "stall_position",
                        "trainer_runs_win_at",
                        "trainer_runs_win_l200r",
                        "trainer_runs_win_l50r",
                        "trainer_runs_win_l16r",
                        "trainer_runs_at",
                        "trainer_runs_l200r",
                        "trainer_runs_l50r",
                        "trainer_runs_l16r",
                        "jockey_runs_win_at",
                        "jockey_runs_win_l200r",
                        "jockey_runs_win_l50r",
                        "jockey_runs_win_l16r",
                        "jockey_runs_at",
                        "jockey_runs_l200r",
                        "jockey_runs_l50r",
                        "jockey_runs_l16r",
                        "horse_runs_win_at",
                        "horse_runs_win_l10r",
                        "horse_runs_win_l5r",
                        "horse_runs_win_l2r",
                        "horse_runs_at",
                        "horse_runs_l10r",
                        "horse_runs_l5r",
                        "horse_runs_l2r",
                        "iv_horse_at",
                        "iv_trainer_l200r",
                        "iv_trainer_l50r",
                        "iv_trainer_l16r",
                        "iv_trainer_at",
                        "iv_jockey_l200r",
                        "iv_jockey_l50r",
                        "iv_jockey_l16r",
                        "iv_jockey_at",
                        "ae_horse_l10r",
                        "ae_horse_l5r",
                        "ae_horse_l2r",
                        "ae_horse_at",
                        "ae_trainer_l200r",
                        "ae_trainer_l50r",
                        "ae_trainer_l16r",
                        "ae_trainer_at",
                        "ae_jockey_l200r",
                        "ae_jockey_l50r",
                        "ae_jockey_l16r",
                        "ae_jockey_at",
                        "rolling_avg_trainer_finish_at",
                        "rolling_avg_trainer_finish_l200r",
                        "rolling_avg_trainer_finish_l50r",
                        "rolling_avg_trainer_finish_l16r",
                        "rolling_avg_horse_finish_at",
                        "rolling_avg_horse_finish_l10r",
                        "rolling_avg_horse_finish_l5r",
                        "rolling_avg_horse_finish_l2r",
                        "rolling_avg_jockey_finish_at",
                        "rolling_avg_jockey_finish_l200r",
                        "rolling_avg_jockey_finish_l50r",
                        "rolling_avg_jockey_finish_l16r",
                        ]
    numeric_transformer = Pipeline(
        steps=[("scaler", MinMaxScaler())])
    print("✅ NUMERIC FEATURES MINMAX-SCALED")

    #--------------------------------------------------------------------------#

    ### IMPUTE HEADGEAR NULLS WITH 'no_headgear' ###
    headgear_feature = ["f_headgear"]
    headgear_imputer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy = 'constant',
                                         fill_value = 'no_headgear'))])
    print("✅ IMPUTED 'no_headgear' for NULLS IN 'f_headgear'")

    #--------------------------------------------------------------------------#

    ### MEAN IMPUTE CERTAIN FEATURES ###
    mean_impute_features = ["f_dob", "f_prb_avg"]
    mean_imputer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy = 'mean'))])
    print("✅ IMPUTED MEAN FOR NULLS IN 'f_dob' & 'f_prb_avg'")

    #--------------------------------------------------------------------------#

    ### IMPUTE NULLS WITH 0's ###
    zero_impute_features = ["f_rating_rbd",
                            "trainer_runs_win_at",
                            "trainer_runs_win_l200r",
                            "trainer_runs_win_l50r",
                            "trainer_runs_win_l16r",
                            "trainer_runs_at",
                            "trainer_runs_l200r",
                            "trainer_runs_l50r",
                            "trainer_runs_l16r",
                            "jockey_runs_win_at",
                            "jockey_runs_win_l200r",
                            "jockey_runs_win_l50r",
                            "jockey_runs_win_l16r",
                            "jockey_runs_at",
                            "jockey_runs_l200r",
                            "jockey_runs_l50r",
                            "jockey_runs_l16r",
                            "horse_runs_win_at",
                            "horse_runs_win_l10r",
                            "horse_runs_win_l5r",
                            "horse_runs_win_l2r",
                            "horse_runs_at",
                            "horse_runs_l10r",
                            "horse_runs_l5r",
                            "horse_runs_l2r",
                            "iv_horse_at",
                            "iv_trainer_l200r",
                            "iv_trainer_l50r",
                            "iv_trainer_l16r",
                            "iv_trainer_at",
                            "iv_jockey_l200r",
                            "iv_jockey_l50r",
                            "iv_jockey_l16r",
                            "iv_jockey_at",
                            "ae_horse_l10r",
                            "ae_horse_l5r",
                            "ae_horse_l2r",
                            "ae_horse_at",
                            "ae_trainer_l200r",
                            "ae_trainer_l50r",
                            "ae_trainer_l16r",
                            "ae_trainer_at",
                            "ae_jockey_l200r",
                            "ae_jockey_l50r",
                            "ae_jockey_l16r",
                            "ae_jockey_at",
                            "rolling_avg_trainer_finish_at",
                            "rolling_avg_trainer_finish_l200r",
                            "rolling_avg_trainer_finish_l50r",
                            "rolling_avg_trainer_finish_l16r",
                            "rolling_avg_horse_finish_at",
                            "rolling_avg_horse_finish_l10r",
                            "rolling_avg_horse_finish_l5r",
                            "rolling_avg_horse_finish_l2r",
                            "rolling_avg_jockey_finish_at",
                            "rolling_avg_jockey_finish_l200r",
                            "rolling_avg_jockey_finish_l50r",
                            "rolling_avg_jockey_finish_l16r"]
    zero_imputer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy = 'constant',
                                         fill_value = 0))])
    print("✅ IMPUTED '0' FOR NULLS IN 68 x FEATURES")

    #--------------------------------------------------------------------------#

    ### O.H.E. CATEGORICAL FEATURES ###
    categorical_features = ["f_track", "f_headgear", "country"]
    categorical_transformer = Pipeline(
        steps=[("encoder", OneHotEncoder(handle_unknown="ignore",
                                     sparse_output=False))])
    print("✅ CAT. FEATURES OH-ENCODED (Track, Headgear, Country)")

    #--------------------------------------------------------------------------#

    ### COLUMN TRANSFORMER ###
    ct1 = ColumnTransformer(
        transformers=[("zero_imputer", zero_imputer, zero_impute_features),
                      ("headgear_imputer", headgear_imputer, headgear_feature),
                      ("mean_imputer", mean_imputer, mean_impute_features),
                      ],
        verbose_feature_names_out = False,
        remainder = 'passthrough')

    ct1_processed = ct1.fit_transform(data)

    print('number of columns: ', len(ct1_processed.columns))

    ct2 = ColumnTransformer(
        transformers=[("cat", categorical_transformer, categorical_features)],
        verbose_feature_names_out = False,
        remainder = 'passthrough')

    ct2_processed = ct2.fit_transform(ct1_processed)

    ct3 = ColumnTransformer(
        transformers=[("scale", numeric_transformer, numeric_features),],
        verbose_feature_names_out = False,
        remainder = 'passthrough')

    print("✅ COLUMN TRANSFORMER ASSEMBLED")

    #--------------------------------------------------------------------------#

    ### FIT_TRANSFORM FEATURES ###
    print("⏳ FIT_TRANSFORMING THE PREPROCESSING PIPE...")
    data_processed = ct3.fit_transform(ct2_processed)
    print('number of columns: ', len(data_processed.columns))
    print("✅ DATA PROCESSED WITH SHAPE:", data_processed.shape)

    return data_processed

In [4]:
data = preprocess_features_v2(data)

number of columns:  127
✅ DROPPED IRRELEVANT COLUMNS
✅ DROPPED ROWS WITH NULL VALUES
✅ WHITESPACE STRIPPED FROM 'f_track'
✅ ODDS CONVERTED TO PROBABILITY (1/ODDS)
✅ WINNERS CODED AS '1', REST '0'
✅ 'f_ko' CONVERTED TO DATETIME
✅ TRACK CONDITIONS ORDINALLY ENCODED
✅ NUMERIC FEATURES MINMAX-SCALED
✅ IMPUTED 'no_headgear' for NULLS IN 'f_headgear'
✅ IMPUTED MEAN FOR NULLS IN 'f_dob' & 'f_prb_avg'
✅ IMPUTED '0' FOR NULLS IN 68 x FEATURES
✅ CAT. FEATURES OH-ENCODED (Track, Headgear, Country)
number of columns:  87
✅ COLUMN TRANSFORMER ASSEMBLED
⏳ FIT_TRANSFORMING THE PREPROCESSING PIPE...
number of columns:  163
✅ DATA PROCESSED WITH SHAPE: (117467, 163)


In [5]:
data.isnull().sum()

f_going                                0
average_or_rating_class                0
above_below_official_rating_class      0
PreviousPosition                       0
PredictedRank                          0
                                    ... 
f_horse                                0
pred_isp                               0
f_place                                0
f_pm_01m                               0
f_pm_01m_p_back                      145
Length: 163, dtype: int64

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
data = data.reset_index(drop=True)

In [14]:
backtest = data[['f_ko','f_id', 'id','f_horse','f_pm_01m', 'f_pm_01m_p_back']]

In [15]:
y = data['f_place']

In [16]:
X = data.drop(columns=['f_pm_01m', 'f_pm_01m_p_back' , 'f_place', 'f_id', 'id', 'f_horse'])

In [17]:
print(data.shape)
print(X.shape)
print(y.shape)
print(backtest.shape)

(117467, 163)
(117467, 157)
(117467,)
(117467, 6)


In [ ]:
row_5 = data.iloc[5]

In [ ]:
row5to10 = data.iloc[5:10]

In [ ]:
row5to10

In [ ]:
data['f_ko']

Data start: Row 0: 2021-03-06 01:25:00. 
Start of year 2: Row 45648:  2022-03-07 17:30:00.    
Start of year 3: Row 91429: 2023-03-06 16:55:00  

In [19]:
X_train = X.iloc[:45648]
X_val = X.iloc[45648:91429]
X_test = X.iloc[91429:]
y_train = y.iloc[:45648]
y_val = y.iloc[45648:91429]
y_test = y.iloc[91429:]
backtest_train = backtest.iloc[:45648]
backtest_val = backtest.iloc[45648:91429]
backtest_test = backtest.iloc[91429:]

In [20]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)
print(backtest_train.shape)
print(backtest_val.shape)
print(backtest_test.shape)

(45648, 157)
(45781, 157)
(26038, 157)
(45648,)
(45781,)
(26038,)
(45648, 6)
(45781, 6)
(26038, 6)


In [21]:
from sklearn.decomposition import PCA
X_train_PCA = X_train.drop(columns=['f_ko'])


In [22]:
X_train_PCA.head(2)

,f_going,average_or_rating_class,above_below_official_rating_class,PreviousPosition,PredictedRank,f_distance,f_class,f_age,f_pace,f_weight,f_runners,f_rating_or,mean_f_rating_or_race,or_rating_vs_avg_race,f_rating_rbd,f_stall,stall_position,trainer_runs_win_at,trainer_runs_win_l200r,trainer_runs_win_l50r,trainer_runs_win_l16r,trainer_runs_at,trainer_runs_l200r,trainer_runs_l50r,trainer_runs_l16r,jockey_runs_win_at,jockey_runs_win_l200r,jockey_runs_win_l50r,jockey_runs_win_l16r,jockey_runs_at,jockey_runs_l200r,jockey_runs_l50r,jockey_runs_l16r,horse_runs_win_at,horse_runs_win_l10r,horse_runs_win_l5r,horse_runs_win_l2r,horse_runs_at,horse_runs_l10r,horse_runs_l5r,horse_runs_l2r,iv_horse_at,iv_trainer_l200r,iv_trainer_l50r,iv_trainer_l16r,iv_trainer_at,iv_jockey_l200r,iv_jockey_l50r,iv_jockey_l16r,iv_jockey_at,ae_horse_l10r,ae_horse_l5r,ae_horse_l2r,ae_horse_at,ae_trainer_l200r,ae_trainer_l50r,ae_trainer_l16r,ae_trainer_at,ae_jockey_l200r,ae_jockey_l50r,ae_jockey_l16r,ae_jockey_at,rolling_avg_trainer_finish_at,rolling_avg_trainer_finish_l200r,rolling_avg_trainer_finish_l50r,rolling_avg_trainer_finish_l16r,rolling_avg_horse_finish_at,rolling_avg_horse_finish_l10r,rolling_avg_horse_finish_l5r,rolling_avg_horse_finish_l2r,rolling_avg_jockey_finish_at,rolling_avg_jockey_finish_l200r,rolling_avg_jockey_finish_l50r,rolling_avg_jockey_finish_l16r,f_track_ASCOT,f_track_AYR,f_track_BALLINROBE,f_track_BATH,f_track_BELLEWSTOWN,f_track_BEVERLEY,f_track_BRIGHTON,f_track_CARLISLE,f_track_CATTERICK,f_track_CHELMSFORD CITY,f_track_CHEPSTOW,f_track_CHESTER,f_track_CLONMEL,f_track_CORK,f_track_CURRAGH,f_track_DONCASTER,f_track_DOWN ROYAL,f_track_DUNDALK,f_track_EPSOM,f_track_FAIRYHOUSE,f_track_FFOS LAS,f_track_GALWAY,f_track_GOODWOOD,f_track_GOWRAN PARK,f_track_HAMILTON,f_track_HAYDOCK,f_track_KEMPTON,f_track_KILLARNEY,f_track_LAYTOWN,f_track_LEICESTER,f_track_LEOPARDSTOWN,f_track_LIMERICK,f_track_LINGFIELD,f_track_LISTOWEL,f_track_MUSSELBURGH,f_track_NAAS,f_track_NAVAN,f_track_NEWBURY,f_track_NEWCASTLE,f_track_NEWMARKET,f_track_NOTTINGHAM,f_track_PONTEFRACT,f_track_PUNCHESTOWN,f_track_REDCAR,f_track_RIPON,f_track_ROSCOMMON,f_track_SALISBURY,f_track_SANDOWN,f_track_SLIGO,f_track_SOUTHWELL,f_track_THIRSK,f_track_THURLES,f_track_TIPPERARY,f_track_TRAMORE,f_track_WETHERBY,f_track_WINDSOR,f_track_WOLVERHAMPTON,f_track_YARMOUTH,f_track_YORK,f_headgear_B,f_headgear_BC,f_headgear_C,f_headgear_EB,f_headgear_ET,f_headgear_H,f_headgear_HB,f_headgear_HC,f_headgear_HE,f_headgear_HT,f_headgear_HV,f_headgear_T,f_headgear_TB,f_headgear_TC,f_headgear_TV,f_headgear_V,f_headgear_VC,f_headgear_no_headgear,country_GB,country_IRE,f_dob,f_prb_avg,pred_isp
0,0.4,0.836105,0.643877,0.0,0.000000,0.0,0.166667,0.25,0.7,0.21875,0.15625,0.703704,0.850515,0.597311,0.916667,0.027778,0.148019,0.0,0.0,0.0,0.0,0.001919,0.025,0.10,0.3125,0.004219,0.019231,0.052632,0.125,0.001053,0.01,0.04,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.499485,0.499484,0.5,0.478974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014276,0.014276,0.014276,0.014276,0.231883,0.231883,0.231883,0.231883,0.0,0.0,0.0,0.0,0.142857,0.142857,0.142857,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.404181,0.434968,0.400000
1,0.4,0.836105,0.691321,0.0,0.032258,0.0,0.166667,0.50,0.5,0.34375,0.15625,0.755556,0.850515,0.645049,0.845238,0.138889,0.480479,0.0,0.0,0.0,0.0,0.000768,0.010,0.04,0.1250,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.195652,0.195652,0.195652,0.195652,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [27]:
pca = PCA(n_components=3)
pca.fit(X_train_PCA)

PCA(n_components=3)

In [32]:
horse_features = X_train_PCA.columns

# Access our 13 PCs 
W = pca.components_

# Print PCs as COLUMNS
W = pd.DataFrame(W.T,
                 index=horse_features,
                 columns=[f'PC{i}' for i in range(1, 4)])
W

,PC1,PC2,PC3
f_going,-0.002615,-0.001002,-3.359594e-03
average_or_rating_class,-0.030949,-0.137406,1.955588e-01
above_below_official_rating_class,-0.001655,-0.005980,7.887135e-03
PreviousPosition,-0.052934,0.050810,-1.023509e-01
PredictedRank,0.026406,0.036067,-4.700640e-02
f_distance,0.015453,-0.006544,2.256971e-02
f_class,0.032810,0.146885,-2.085596e-01
f_age,-0.001004,0.055840,-3.749721e-02
f_pace,-0.010624,-0.027655,4.806129e-02
f_weight,-0.003942,0.003680,-4.907922e-03


In [31]:
pd.set_option('display.max_rows', None)